In [2]:
import import_ipynb
from ElektronImFeld import ElektronImFeld
from EmptySmarti import EmptySmarti
from Smartis import *   
from Buttons import *   
from Chart2 import *
from ipywidgets import HBox, VBox, Layout
from math import pi, sin, cos
from IPython.display import display, clear_output

class Replay(Smartis):
    
    def __init__(self,parameterList):
        self.smartiDict ={"ElektronImFeld":ElektronImFeld,"EmptySmarti":EmptySmarti}
        self.smartiObjectDict = {}
        super().__init__(parameterList)
        self.stopSendInformation()
        self.updateTime = 0.5
        self.replayIndex = 0
        self.dataArray = []
        
        
        
    def createButtons(self):        
        
        #Erzeuge das Object, welches alle Buttons managt
        self.buttonsObject = Buttons(self)        
        
        #Erzeuge verschiedene Buttons
        
        dropdown_smarti = self.buttonsObject.newDropdown("dd_smarti", [], "SmartI:")
        dropdown_userID = self.buttonsObject.newDropdown("dd_userID", [], "UserID:")
        dropdown_runNumber = self.buttonsObject.newDropdown("dd_runNumber", [], "Durchlauf:")
        button_runNumber = self.buttonsObject.newTogglePlayPause("b_runNumber", 30,30)
        

     
        #Aktualisiere ein Dictionarie mit den Stati der Buttons        
        self.buttonStatesDict = self.buttonsObject.getStates()
        self.oldButtonStatesDict = dict(self.buttonStatesDict)
        
        self.initalizeSmartiWidget()
        self.initalizeUserIDWidget()
        self.initalizeRunNumberWidget()
        
        self.buttonsDict = self.buttonsObject.getButtons()
            
    def createChart(self):

        for name in self.smartiNames:            
            s = name.split("-")
            smarti = s[0]
            arg = s[1].replace("(", "").replace(")", "")
            
            newSmarti = self.smartiDict[smarti](arg)    
            newSmarti.stopSendInformation()

            newButtonObject = newSmarti.getButtonObject()
            newButtons = newButtonObject.getButtons()

            for key in newButtons:
                #print(self.currentButtons[key])
                newButtonObject.disabled(key,True)
            print(name)
            self.smartiObjectDict[name] = newSmarti

        self.getCurrentSmarti()
            
    def visualize(self): 
        
        clear_output()        
        
        #Darstellung   
        display(HBox([self.buttonsDict["dd_smarti"],self.buttonsDict["dd_userID"],
                      self.buttonsDict["dd_runNumber"],self.buttonsDict["b_runNumber"]]))
        self.currentSmarti.visualize()

    
    
    def getCurrentSmarti(self):
        
        self.currentSmarti = self.smartiObjectDict[self.buttonStatesDict["dd_smarti"]]
            
        self.currentButtonObject = self.currentSmarti.getButtonObject()
        self.currentButtons = self.currentButtonObject.getButtons()
            
        

     
    def update(self):
        
        if self.oldButtonStatesDict["dd_smarti"]!= self.buttonStatesDict["dd_smarti"]:
            self.getCurrentSmarti()
            self.visualize()
            self.initalizeUserIDWidget()
            self.initalizeRunNumberWidget()
        if self.oldButtonStatesDict["dd_userID"]!= self.buttonStatesDict["dd_userID"]:
            self.initalizeRunNumberWidget()
        if self.oldButtonStatesDict["b_runNumber"]!= self.buttonStatesDict["b_runNumber"]:
            if self.buttonStatesDict["dd_runNumber"]!= None:
                self.dataArray = self.sessionData[self.buttonStatesDict["dd_runNumber"]]
                
                for key in self.buttonStatesDict:            
                    self.buttonsObject.disabled(key,True)
                self.currentSmarti.start()
                self.startReplay()
                self.buttonsObject.changeStates("b_runNumber",False)
            else:
                self.buttonsObject.changeStates("b_runNumber",False)
    

        


        
        
        
    def initalizeSmartiWidget(self):
        self.smartiNames = Server.getAllSmartiNames()
        self.buttonsObject.changeOptions("dd_smarti",self.smartiNames)
     
    def initalizeUserIDWidget(self):
        userID = Server.getUserIDofSmarti(self.buttonStatesDict["dd_smarti"])
        self.buttonsObject.changeOptions("dd_userID",userID)
        
    def initalizeRunNumberWidget(self):
        #print(self.sessionData)
        self.sessionData = Server.getSession(self.buttonStatesDict["dd_smarti"], self.buttonStatesDict["dd_userID"])
        if len(self.sessionData)>=1:
            numbers = list(range(len(self.sessionData)))
        else:
            numbers = []
            
        self.buttonsObject.changeOptions("dd_runNumber",numbers)
        
    def startReplay(self):
        #Array Positions:
        userIDPosition = 0
        smartiPosition = 1
        smartiArgumentsPosition = 2
        timePosition = 3
        buttonPosition = 4
        valuePosition = 5
        dataLength = len(self.dataArray)
        i = self.replayIndex
        currentData = self.dataArray[i]
     
        self.currentButtonObject.changeStates(currentData[buttonPosition],self.castingValue(currentData[valuePosition]))
        #print(currentData[buttonPosition],self.castingValue(currentData[valuePosition]))
        currentTime = datetime.strptime(currentData[timePosition], "%m/%d/%Y, %H:%M:%S.%f")-timedelta(seconds=0)
        
        if i < dataLength-1:
            nextTime = datetime.strptime(self.dataArray[i+1][timePosition], "%m/%d/%Y, %H:%M:%S.%f")-timedelta(seconds=0)
            timeDiff = nextTime-currentTime
            #print(timeDiff.total_seconds())    
            threading.Timer(round(timeDiff.total_seconds(),3), self.startReplay).start()
            self.replayIndex+=1
        else:
            self.replayIndex = 0
            for key in self.buttonStatesDict:            
                self.buttonsObject.disabled(key,False)
                self.currentSmarti.stop()
            
            
            
        
     
    def castingValue(self,value):
        if value == "True" or value == "False":
            return bool(value)
        else:
            try:
                a = float(value)
                b = int(value)
                if a != b:
                    return a
                else:
                    return b
            except:
                #raise ValueError("failed to coerce str to int or float")
                return value
            
    def clearFile(self):
        Server.clearFile()

           
        
r = Replay(None)
r.start()
r.visualize()    
            

In [2]:
r.clearFile()


In [5]:
>>> class Circle():
...     pass
... 
>>> class Square():
...     pass
... 
>>> d = { "square": Square, "circle": Circle}
>>> d["square"]()

In [3]:
e.stop()